#### Define settings

In [1]:
sample_n = 10
replica_n = 2
# the number of chosen samples
top_n = 2

import json
SCHEMES_PATH = "schemes.json"
with open(SCHEMES_PATH) as schemes_file:
    schemes = json.load(schemes_file)["schemes"]
    
run_flag = False

##### Define free parameters

In [2]:
free_params = {
#     "AE_H_t": [0,1],
#     "AE_L_t": [0,1],
#     "B_MSC_rec": [0.001,0.005],
    "B_MSC_Pr": [0.01,0.05],
#     "B_MSC_Mo": [0.0015,0.01],
    "CD_H_t": [0.5,1],
    "CD_L_t": [0,0.5]
#     "CD_M_t1": 0.3,
#     "CD_M_t2": 0.6,
#     "MG_H_t": [15,40],
#     "MG_L_t1": [0,10],
#     "MG_L_t2": [3,15],
#     "Mo_H_v": [2,5],
#     "Pr_N_v": [0,1],
#     "w_lactate_ph": [0.1,0.5],
#     "w_mg_ph": [0.02,0.1],
#     "w_MI_lactate": [0.05,0.1]
}
free_params_keys = list(free_params.keys())
free_params_bounds = list(free_params.values())


In [3]:

def distance_function(realizations,expectations):
    import numpy as np
    distances = []
    for i in range(len(realizations)):
        raw_sim_data = realizations[i]
        sim_data = {}
        sim_liveCellCount = np.array(raw_sim_data["agents_count"]["MSC"])
        sim_totalCellCount = np.array(raw_sim_data["agents_count"]["MSC"]) + np.array(raw_sim_data["agents_count"]["Dead"])
        sim_viability = sim_liveCellCount[-1]/sim_totalCellCount[-1] 
        sim_data.update({"liveCellCount":sim_liveCellCount,
                         "viability": sim_viability})
                
        exp_data = expectations[i]

        error_liveCellCount = (sim_data["liveCellCount"] - np.array(exp_data["liveCellCount"]))/np.array(exp_data["liveCellCount"])
        error_viability = (sim_data["viability"] - np.array(exp_data["viability"]))/np.array(exp_data["viability"])

        all_errors = np.concatenate((error_liveCellCount,error_viability),axis=0)
        abs_all_errors = np.abs(all_errors)

        distances.append(np.mean(abs_all_errors)) 
    
    distance = np.mean(distances)
    

    return distance

In [4]:
import sys
sys.path.insert(1, '/Users/matin/Downloads/testProjs/CA')
import time
import numpy as np
class clock:
    start_t = 0
    end_t = 0
    @staticmethod
    def start():
        clock.start_t = time.time()
    @staticmethod
    def end():
        clock.end_t = time.time()
        print('Elapsed time: ',clock.end_t - clock.start_t)

import numpy as np
class Process:
    def __init__(self,free_params,sample_n):
        self.free_params = free_params
        self.free_params_keys = list(free_params.keys())
        self.free_params_bounds = list(free_params.values())
        self.sample_n = sample_n


    def sample(self):
        from diversipy import lhd_matrix
        from diversipy import transform_spread_out
        import json
        # python version > 3.6
        non_scalled_samples = transform_spread_out(lhd_matrix(self.sample_n, len(self.free_params))).transpose()
        scaled_samples = []
        ii = 0
        for bounds in self.free_params_bounds:
            low = bounds[0]
            high = bounds[1]
            
            pre_samples_param = non_scalled_samples[ii]
            
            samples_param = list(map(lambda x:x*(high-low)+low ,pre_samples_param))
            scaled_samples.append(samples_param)
            ii+=1
        priors = {key:value for key,value in zip(self.free_params_keys,scaled_samples)}
        samples = np.array(scaled_samples).transpose()
        np.savetxt('outputs/samples.txt', samples, fmt='%f')
        ##### create parameter sets
        param_sets = []
        for sample in samples:
            param_set = {}
            for i in range(len(sample)):
                sample_p = sample[i]
                key = self.free_params_keys[i]
                param_set.update({key:sample_p})
            param_sets.append(param_set)
        with open("outputs/param_sets.json",'w') as file:
            file.write(json.dumps({"param_sets":param_sets}))

        self.param_sets = param_sets


    param_sets = 0
    sample_n = 0
    def run_model(self,start_end):
        import copy

        start_n = start_end[0]
        end_n = start_end[1]
        distances = np.array([])
    #     bar.start()
        for i in range(start_n,end_n):
            param_set = self.param_sets[i]
    #         print("iteration ",i)
            schemes_copy = copy.deepcopy(self.schemes)
            rep_distances = []
            for rep_i in range(self.replica_n):
                try:
                    sim_results_list = self.Model(param_set).run(schemes_copy)
                except ValueError:
                    # None value if the param set leads to invalid definition
                    distances = np.append(distances,None) 
                    break
                distance = self.distance_function(sim_results_list,self.expectations)
                rep_distances.append(distance)
            else:
    #             print(rep_distances)
                distances = np.append(distances,np.mean(rep_distances))
    #         bar.update(i+1)
    #     bar.finish()
        return distances
    expectations = 0    
    replica_n = 0
    Model = 0
    distance_function = 0
    schemes = 0
    def run(self,CPU_n,Model, distance_function, schemes, replica_n):
        import numpy as np
        self.expectations = [scheme["expectations"] for scheme in schemes]
        self.replica_n = replica_n
        self.Model = Model
        self.distance_function = distance_function
        self.schemes = schemes
        from multiprocessing import Pool
        CPU_n = 4

        main_share = int(self.sample_n/CPU_n)
        plus =  self.sample_n%CPU_n
        indices =[np.array([x,x+1])*main_share for x in range(CPU_n)] 
        for i in range(plus):
            indices[i][1]+=1

        pl = Pool(CPU_n)
        clock.start()
        results = pl.map(self.run_model,indices)
        clock.end()


        distances = [] 
        for result in results:
            distances = np.concatenate([distances,result],axis=0)

        np.savetxt('outputs/distances.txt',np.array(distances),fmt='%s')

from model import Model
sys.path.insert(1, './src/')
from process import Process
if run_flag:
    pr = Process(free_params=free_params,sample_n=sample_n)
    pr.sample()
    pr.run(CPU_n=CPU_n, Model=Model,distance_function=distance_function,schemes=schemes,replica_n = replica_n)

## Post processing:


### settings

In [5]:
plot_flag = False


#### Recover the data 

In [6]:
import numpy as np
distances = []
with open("outputs/distances.txt") as file:
    for line in file:
        line.strip()
        try:
            value = float(line)
        except:
            value = None
        distances.append(value)
samples = np.loadtxt('outputs/samples.txt', dtype=float)

In [7]:
### Fitness values and plots

In [8]:
import json
import numpy as np
fitness_values = np.array([])
for item in distances:
    if item == None:
        fitness = 0
    else:
        fitness = 1 - item
    fitness_values = np.append(fitness_values,fitness)
top_ind = np.argpartition(fitness_values, -top_n)[-top_n:]
top_fitess_values = fitness_values[top_ind]
np.savetxt("outputs/top_fitness.txt",top_fitess_values,fmt='%f')


### Calculate posteriors
- Extract the indices of min n values of distances
- Extract the best fits
- Extract associated samples
- Create posteriors
- Save posteriors

In [9]:
import json

top_fit_samples = samples[top_ind]
posteriors = {key:list(value) for key,value in zip(free_params_keys,top_fit_samples.transpose())}
with open('outputs/posterior.json', 'w') as file:
     file.write(json.dumps({'posteriors': posteriors}))
# max(posteriors["B_MSC_Pr"])


### scale the posterior

In [10]:
scalled_posteriors = {}
for key,values in posteriors.items():
    min_v = free_params[key][0]
    max_v = free_params[key][1]
    scalled = list(map(lambda x: (x-min_v)/(max_v-min_v),values))
    scalled_posteriors.update({key:scalled})

### Box Plot: median, quartiles, and raw data

In [11]:
import plotly.graph_objects as go
import plotly.offline
def plot():
    colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)',
              'rgba(255, 65, 54, 0.5)', 'rgba(207, 114, 255, 0.5)', 'rgba(127, 96, 0, 0.5)']
    traces = []
    ii = 0
    for key,value in scalled_posteriors.items():
        traces.append(go.Box(
            y=value,
            name=key,
            boxpoints='all',
            jitter=0,
            fillcolor=colors[ii],
            marker_size=5,
            whiskerwidth=0.2,
            line_width=2)
                     )
        ii += 1
    layout = go.Layout(yaxis=dict(
    #                             autorange=True,
    #                             showgrid=False,
                                dtick=0.2,
                                zeroline = False,range= [-0.1,1.1]
                                ),
                        margin=dict(
                                l=40,
                                r=30,
                                b=80,
                                t=100
                            ),
                        showlegend=False,
                        paper_bgcolor='rgb(243, 243, 243)',
                        plot_bgcolor='rgb(243, 243, 243)',
                       )
    fig = { "data": traces,"layout":layout }
    plotly.io.write_image(fig = { "data": traces,"layout":layout }, file="outputs/box_plot.svg",format="svg",scale=None, width=None, height=None)
    # plotly.offline.plot(fig = { "data": traces,"layout":layout }, auto_open = True)
if plot_flag:
    plot()

### Plot against empirical data

### Run the model for posteriors
The run function should be different in terms of collecting results of for all individual measurement points, running on the posterior param sets

In [12]:
import sys
sys.path.insert(1, '/Users/matin/Downloads/testProjs/CA')
from model import Model
import copy
import numpy as np

def AVERAGE_OVER_REPLICAS(items):
    template = items[0]["agents_count"]
    aver = {}
    for key in template.keys():
        stacks = [replica["agents_count"][key] for replica in items]
        stacks = np.array(stacks)
        mean_v = np.mean(stacks,axis = 0)
        aver.update({key:mean_v})
    aver = {"agents_count":aver}
#     print(aver,'\n')
    return aver
def AVERAGE(replicas_schemes):
    scheme_data = []
    for scheme_i in range(len(schemes)):
        replicas = [replica[scheme_i] for replica in replicas_schemes]
        replicas_averaged = AVERAGE_OVER_REPLICAS(replicas)
        scheme_data.append(replicas_averaged)  
    return scheme_data

def run_model_post(args):
    full_results_repo = []
    start_n = args[0]
    end_n = args[1]
    for i in range(start_n,end_n):
        param_set = post_param_sets[i]
        schemes_copy = copy.deepcopy(schemes)
        sim_results_list_replicas = []
        for rep_i in range(replica_n_post):
            try:
                sim_results_list = Model(param_set).run(schemes_copy)
            except ValueError:
                print("This cannot happen at this stage")
                sys.exit()
            sim_results_list_replicas.append(sim_results_list)
        else:
            full_results_repo.append(AVERAGE(sim_results_list_replicas))
    return full_results_repo

In [13]:
from multiprocessing import Pool
import numpy as np
import json
CPU_n = 4
replica_n_post = 2

with open("outputs/param_sets.json") as file:
    param_sets = json.load(file)["param_sets"]
    
post_param_sets = np.array(param_sets)[top_ind]
main_share = int(len(post_param_sets)/CPU_n)
plus =  len(post_param_sets)%CPU_n
indices =[np.array([x,x+1])*main_share for x in range(CPU_n)] 
for i in range(plus):
    indices[i][1]+=1
    
pl = Pool(CPU_n)
results = pl.map(run_model_post,indices)

# Pile up the results
full_results_repo = np.array([])
for result in results:
    if len(result) == 0:
        continue
    if len(full_results_repo) == 0:
        full_results_repo = result
    else:
        full_results_repo = np.concatenate([full_results_repo,result],axis=0)

{'agents_count': {'Dead': array([10.5, 23. , 47. ]), 'MSC': array([157. , 269.5, 371. ])}} 

{'agents_count': {'Dead': array([ 8. , 25. , 53.5]), 'MSC': array([167.5, 309. , 389. ])}} 

{'agents_count': {'Dead': array([10., 23., 54.]), 'MSC': array([165.5, 287. , 373.5])}} 

{'agents_count': {'Dead': array([ 6.5, 20. , 61.5]), 'MSC': array([169. , 328. , 389.5])}} 



In [15]:
full_results_repo

array([[{'agents_count': {'Dead': array([10., 23., 54.]), 'MSC': array([165.5, 287. , 373.5])}},
        {'agents_count': {'Dead': array([ 6.5, 20. , 61.5]), 'MSC': array([169. , 328. , 389.5])}}],
       [{'agents_count': {'Dead': array([10.5, 23. , 47. ]), 'MSC': array([157. , 269.5, 371. ])}},
        {'agents_count': {'Dead': array([ 8. , 25. , 53.5]), 'MSC': array([167.5, 309. , 389. ])}}]],
      dtype=object)

In [18]:
expectations = [scheme["expectations"] for scheme in schemes]
expectations

[{'liveCellCount': [120, 260, 370], 'viability': [0.95]},
 {'liveCellCount': [120, 240, 470], 'viability': [0.93]}]

In [28]:
import random
data1 = []
data2 = []
for i in range(100):
    vv = random.random()
    data1.append(vv)
    data2.append(.2*vv)
print(min(data1),max(data1))
print(min(data2),max(data2))
from scipy.stats import levene
levene(data1,data2)

0.006296540126285333 0.9816881000486329
0.0012593080252570666 0.1963376200097266


LeveneResult(statistic=155.75629968290164, pvalue=9.475667695337911e-27)